In [ ]:
from google.colab import files

!pip install -q -U pandas
import pandas as pd
import io

In [ ]:
try:
  df = pd.read_csv('Data_Entry_2017.csv')
except:
  uploaded = files.upload()
  df = pd.read_csv(io.BytesIO(uploaded['Data_Entry_2017.csv']))

df.head(n=1)

Saving Data_Entry_2017.csv to Data_Entry_2017.csv


,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],Unnamed: 11
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,NaN


## Pick Patients who have at least 3 followups (indexing from 0)

In [ ]:
pd.set_option('display.max_columns', None)
df_updated_followups = df.groupby('Patient ID').filter(lambda x : len(x)>2)
print("Filter 1")
print("Patients with at least 3 follow-ups: "+ str(df_updated_followups.groupby('Patient ID').ngroups))
print("-----------------------------------------------")
'''
remove patients that have followups with different view positions.
if patient 1 has PA and AP images then remove patient 1
then save filtered data into a csv named df_updated_view_position.csv
'''
df_updated_view_postion = df_updated_followups[df_updated_followups.groupby("Patient ID")["View Position"].transform('nunique')<=1]
df_updated_view_postion = df_updated_view_postion.reset_index()
print("Filter 2")
print("Patients homogenous view positions:" +str(df_updated_view_postion.groupby('Patient ID').ngroups))



Filter 1
Patients with at least 3 follow-ups: 9189
-----------------------------------------------
Filter 2
Patients homogenous view positions:2992


#### Splitting rows if more than 1 finding label for follow up 2 and above

In [ ]:
import csv ,os

df_updated_view_postion.to_csv('df_updated_view_postion.csv',na_rep='=NA()')

**Example 1**

        Sample set should consist of 3 follow-ups. In each sample set we look 
        at the 3rd follow-up. If Followup 3 has more than 1 disease then we 
        will create a new sample containing identical follow-ups 0 and 1 but 
        there will be a new third follow-up depending on the disease.

        example: 
          00000013_023.png , Infiltration|Mass|Pneumothorax        Follow-up 0, Patient 13
          00000013_024.png , Mass                                  Follow-up 1 , Patient 13
          00000013_025.png , Cardiomegaly|Infiltration             Follow-up 2 , Patient 13

        result :
          00000013_023.png , Infiltration|Mass|Pneumothorax        Follow-up 0, Patient 13, Sample # 1
          00000013_024.png , Mass                                  Follow-up 1 , Patient 13, Sample # 1
          00000013_025.png , Cardiomegaly                          Follow-up 2,Patient 13, Sample # 1

          00000013_023.png , Infiltration|Mass|Pneumothorax        Follow-up 0, Patient 13, Sample # 2
          00000013_024.png , Mass                                  Follow-up 1,Patient 13, Sample # 2
          00000013_025.png , Infiltration                          Follow-up 2,Patient 13, Sample # 2


In [ ]:
# Read in filtered data csv 
try:
  new_df = pd.read_csv('df_updated_view_postion.csv')
except:
  uploaded = files.upload()
  new_df = pd.read_csv(io.BytesIO(uploaded['df_updated_view_postion.csv']))

# remove index columns from csv
new_df = new_df.iloc[:,2:13]

# list of diseases to search through
find_labels=["Atelectasis","Consolidation","Infiltration","Pneumothorax","Edema","Emphysema","Fibrosis","Effusion","Pneumonia","Pleural_Thickening","Cardiomegaly","Nodule","Mass","Hernia"]
label = {}

# get the field names to write to new csv. Add Sample # column
fieldnames = new_df.columns.tolist()
fieldnames.append("Sample #")

# keeping track of first follow-up in sample
pointer_followup_init= 0
# keeping track of third follow-up in sample
pointer_followup_final= 0

# Sample set will contain 3 followups
sample_set = []

# keep track of sample number
sample_number=  0

# open new csv to write to named df_updated_finding_labels.csv
with open('df_updated_finding_labels.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile, delimiter=',')

  # write the header names as first row in csv
  writer.writerow(fieldnames)

  for index, row in new_df.iterrows():
    # Look ahead logic to skip to relevant rows when sample sets for patient
    # have been written to new csv
    if index > 0 and index != pointer_followup_init:
      continue

    # set new sample set for upon seeing a new patient
    # and initialize the follow-up 0 and 2 index
    if row['Follow-up #'] == 0:
      pointer_followup_init = index
      pointer_followup_final= pointer_followup_init + 2

    # sample_set will hold the first two follow-ups within a sample set
    sample_set.append(new_df.iloc[pointer_followup_init,:].tolist())
    sample_set[0].append(sample_number)

    sample_set.append(new_df.iloc[pointer_followup_init+1,:].tolist())
    sample_set[1].append(sample_number)

    # check followup 2 in sample set
    # if it contains 1 disease then add it to sample set and write the set to csv
    if '|'  not in new_df.iloc[pointer_followup_final,1]:
        sample_set.append(new_df.iloc[pointer_followup_final,:].tolist())
        sample_set[2].append(sample_number)
        sample_number += 1
        for x1_row in sample_set:
          writer.writerow(x1_row)
        sample_set.clear()
    # otherwise look for diseases in follow 2 in sample set
    else: 
      for disease in find_labels:
        ''' 
        Steps
          1) Add individual disease into sample set and write to csv
          2) Remove follow-up 2 from sample set and rewrite the new disease
          3) Write new sample set while maintaining follow-up 0,follow-up 1
             but adding new follow-up 2 into csv
          4) Do the same till all the diseases are extracted from original follow-up 2
             that contains multiple diseases
          Refer to Example 1
        '''
        #  check diseases in follow-up 2 'Finding Labels' column 
        if disease in new_df.iloc[pointer_followup_final,1]: 
          temp_finding_label = new_df.iloc[pointer_followup_final,1]
          new_df.iloc[pointer_followup_final,1] = disease
          sample_set.append(new_df.iloc[pointer_followup_final,:].tolist())
          new_df.iloc[pointer_followup_final,1] = temp_finding_label
          sample_set[2].append(sample_number)
          for x_row in sample_set:
            # delete sample number of followup within a set
            if not isinstance(x_row,list):
              x_row = list(x_row)
            # keep track of sample number within a set
            x_row[-1] = sample_number
            writer.writerow(x_row)
          
          del sample_set[2]
          sample_number += 1
      sample_set=[]
    # keep track of end of dataframe and exit when end is reached
    if pointer_followup_final + 1 >= new_df.shape[0]:
      break
      
    # update follow-up init pointer which looks ahead for new
    # encountered patient using value in 'Follow-up #' column
    if new_df.iloc[pointer_followup_final+1,2] == 0:
      pointer_followup_init = pointer_followup_final+1
    elif new_df.iloc[pointer_followup_final+1,2] > 0:
      pointer_followup_init += 1
      pointer_followup_final += 1



## Create two different datasets based on view position

### PA Position

In [ ]:
'''
  Read in csv with newly created sample sets 
  Find all the patients with PA View Positions in the Follow-ups
  Rewrite new sample numbers for filtered data 
  Then write results in df_PA.csv
'''

try:
  df_updated_finding_labels = pd.read_csv('df_updated_finding_labels.csv')
except:
  uploaded = files.upload()
  df_updated_finding_labels = pd.read_csv(io.BytesIO(uploaded['df_updated_finding_labels.csv']))

df_PA = df_updated_finding_labels.loc[df_updated_finding_labels['View Position']=="PA",:]
df_PA = df_PA.iloc[:,0:11]


sample_num = 1
count = 0
fieldnames = df_PA.columns.tolist()
fieldnames.append('Sample #')
with open('df_PA.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile, delimiter=',')
  # print(fieldnames)
  writer.writerow(fieldnames)
  for index, row in df_PA.iterrows():
    
    if count == 3:
      sample_num += 1
      count = 0
    count += 1
    row = row.tolist()
    row.append(sample_num)
    writer.writerow(row)


In [ ]:
print("Number Unique Patients in PA dataset: " + str(df_PA.groupby('Patient ID').ngroups))
print("Number of Samples in PA dataset: "+str(int(df_PA.shape[0]/3)))

Number Unique Patients in PA dataset: 2552
Number of Samples in PA dataset: 8114


### AP Position

In [ ]:
'''
  Read in csv with newly created sample sets 
  Find all the patients with AP View Positions in the Follow-ups
  Rewrite new sample numbers for filtered data 
  Then write results in df_AP.csv
'''
try:
  df_updated_finding_labels = pd.read_csv('df_updated_finding_labels.csv')
except:
  uploaded = files.upload()
  df_updated_finding_labels = pd.read_csv(io.BytesIO(uploaded['df_updated_finding_labels.csv']))

df_AP = df_updated_finding_labels.loc[df_updated_finding_labels['View Position']=="AP",:]
df_AP = df_AP.iloc[:,0:11]


sample_num = 1
count = 0
fieldnames = df_AP.columns.tolist()
fieldnames.append('Sample #')
with open('df_AP.csv', 'w', newline='') as csvfile:
  writer = csv.writer(csvfile, delimiter=',')
  # print(fieldnames)
  writer.writerow(fieldnames)
  for index, row in df_AP.iterrows():
    
    if count == 3:
      sample_num += 1
      count = 0
    count += 1
    row = row.tolist()
    row.append(sample_num)
    writer.writerow(row)


In [ ]:
print("Number Unique Patients in AP dataset: " + str(df_AP.groupby('Patient ID').ngroups))
print("Number of Samples in AP dataset: "+str(int(df_AP.shape[0]/3)))

Number Unique Patients in AP dataset: 440
Number of Samples in AP dataset: 3905
